In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import json
import plotly.express as px

from MCForecastTools import MCSimulation


%matplotlib inline

In [3]:
mls_data_path = "./MLS.csv"

In [4]:
mls_data_df = pd.read_csv(mls_data_path, infer_datetime_format=True)

#Previewing Data

mls_data_df.tail()

,Location,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
5086,York Region,287.0,970300.0,11.76,292.0,1112000.0,12.65,293.2,847300.0,12.73,271.3,704100.0,11.28,252.2,562700.0,6.91,2020-11-01
5087,York Region,289.5,978700.0,12.56,295.2,1124200.0,13.89,296.3,856200.0,13.66,270.4,701800.0,10.28,253.2,564900.0,6.84,2020-12-01
5088,York Region,294.9,997000.0,13.34,301.7,1148900.0,15.20,302.9,875300.0,14.78,272.6,707500.0,10.14,254.7,568200.0,5.86,2021-01-01
5089,York Region,309.9,1047700.0,17.16,318.6,1213300.0,20.23,322.6,932200.0,20.15,280.7,728500.0,11.04,260.2,580500.0,5.39,2021-02-01
5090,York Region,324.1,1095700.0,19.82,334.4,1273400.0,23.49,337.2,974400.0,22.48,294.0,763000.0,14.75,268.7,599500.0,6.00,2021-03-01


In [5]:
mls_data_df.count

<bound method DataFrame.count of                Location  CompIndex  CompBenchmark  CompYoYChange  \
0     Adjala-Tosorontio      143.7       510000.0          12.00   
1     Adjala-Tosorontio      140.8       499700.0          13.82   
2     Adjala-Tosorontio      142.7       506400.0          15.83   
3     Adjala-Tosorontio      138.4       491100.0          11.08   
4     Adjala-Tosorontio      145.4       516000.0          16.51   
...                 ...        ...            ...            ...   
5086        York Region      287.0       970300.0          11.76   
5087        York Region      289.5       978700.0          12.56   
5088        York Region      294.9       997000.0          13.34   
5089        York Region      309.9      1047700.0          17.16   
5090        York Region      324.1      1095700.0          19.82   

      SFDetachIndex  SFDetachBenchmark  SFDetachYoYChange  SFAttachIndex  \
0             143.7           510000.0              12.00            NaN  

In [7]:
mls_data_df.count()

Location             5091
CompIndex            5076
CompBenchmark        5076
CompYoYChange        5076
SFDetachIndex        5076
SFDetachBenchmark    5076
SFDetachYoYChange    5075
SFAttachIndex        4949
SFAttachBenchmark    4949
SFAttachYoYChange    4951
THouseIndex          3803
THouseBenchmark      3803
THouseYoYChange      3804
ApartIndex           4010
ApartBenchmark       4010
ApartYoYChange       4008
Date                 5091
dtype: int64

In [6]:
#Viewing Data Types
mls_data_df.dtypes

Location              object
CompIndex            float64
CompBenchmark        float64
CompYoYChange        float64
SFDetachIndex        float64
SFDetachBenchmark    float64
SFDetachYoYChange    float64
SFAttachIndex        float64
SFAttachBenchmark    float64
SFAttachYoYChange    float64
THouseIndex          float64
THouseBenchmark      float64
THouseYoYChange      float64
ApartIndex           float64
ApartBenchmark       float64
ApartYoYChange       float64
Date                  object
dtype: object

In [8]:
mls_data_df['Location'].value_counts()

City of Toronto              138
Adjala-Tosorontio             69
Toronto E05                   69
Toronto E04                   69
Toronto E03                   69
                            ... 
EGswsiallimbury               41
Essa                          15
Bradford West Gwillimbury     15
Barrie                        15
GEswsiallimbury               13
Name: Location, Length: 77, dtype: int64

In [9]:
#There are 77 Regions of Ontario listed in this document

In [10]:
mls_data_df.isnull()

,Location,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
0,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False
1,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False
2,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False
3,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False
4,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5086,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5087,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5088,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5089,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [11]:
mls_data_df.isnull().sum()

Location                0
CompIndex              15
CompBenchmark          15
CompYoYChange          15
SFDetachIndex          15
SFDetachBenchmark      15
SFDetachYoYChange      16
SFAttachIndex         142
SFAttachBenchmark     142
SFAttachYoYChange     140
THouseIndex          1288
THouseBenchmark      1288
THouseYoYChange      1287
ApartIndex           1081
ApartBenchmark       1081
ApartYoYChange       1083
Date                    0
dtype: int64

In [13]:
#Determining percentage of nulls
mls_data_df.isnull().sum() / len(mls_data_df) * 100

Location              0.000000
CompIndex             0.294638
CompBenchmark         0.294638
CompYoYChange         0.294638
SFDetachIndex         0.294638
SFDetachBenchmark     0.294638
SFDetachYoYChange     0.314280
SFAttachIndex         2.789236
SFAttachBenchmark     2.789236
SFAttachYoYChange     2.749951
THouseIndex          25.299548
THouseBenchmark      25.299548
THouseYoYChange      25.279906
ApartIndex           21.233549
ApartBenchmark       21.233549
ApartYoYChange       21.272834
Date                  0.000000
dtype: float64

In [14]:
#Townhouse Data and Apartment Data is absent from over 20% of the regions of Ontario

In [15]:
mls_data_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
5086    False
5087    False
5088    False
5089    False
5090    False
Length: 5091, dtype: bool

In [17]:
#Drop Records that have at least one null value
mls_data_df.dropna(inplace=True)
mls_data_df

,Location,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
69,Ajax,179.8,444300.0,16.00,178.8,478900.0,17.86,187.1,402400.0,15.64,170.5,311600.0,18.16,148.7,252500.0,5.61,2015-07-01
70,Ajax,180.4,445800.0,16.31,179.2,480000.0,15.76,187.6,403500.0,16.81,170.4,311400.0,15.76,151.5,257200.0,8.99,2015-08-01
71,Ajax,182.8,451700.0,15.55,181.4,485800.0,15.10,190.1,408800.0,15.99,172.3,314900.0,14.33,154.3,262000.0,10.53,2015-09-01
72,Ajax,183.8,454200.0,15.74,182.3,488300.0,14.51,191.2,411200.0,15.95,177.1,323700.0,17.13,152.4,258800.0,8.70,2015-10-01
73,Ajax,182.1,450000.0,12.69,180.3,482900.0,11.99,190.3,409300.0,13.14,173.1,316300.0,12.99,151.2,256700.0,5.00,2015-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5086,York Region,287.0,970300.0,11.76,292.0,1112000.0,12.65,293.2,847300.0,12.73,271.3,704100.0,11.28,252.2,562700.0,6.91,2020-11-01
5087,York Region,289.5,978700.0,12.56,295.2,1124200.0,13.89,296.3,856200.0,13.66,270.4,701800.0,10.28,253.2,564900.0,6.84,2020-12-01
5088,York Region,294.9,997000.0,13.34,301.7,1148900.0,15.20,302.9,875300.0,14.78,272.6,707500.0,10.14,254.7,568200.0,5.86,2021-01-01
5089,York Region,309.9,1047700.0,17.16,318.6,1213300.0,20.23,322.6,932200.0,20.15,280.7,728500.0,11.04,260.2,580500.0,5.39,2021-02-01


In [18]:
mls_data_df.isnull().sum()

Location             0
CompIndex            0
CompBenchmark        0
CompYoYChange        0
SFDetachIndex        0
SFDetachBenchmark    0
SFDetachYoYChange    0
SFAttachIndex        0
SFAttachBenchmark    0
SFAttachYoYChange    0
THouseIndex          0
THouseBenchmark      0
THouseYoYChange      0
ApartIndex           0
ApartBenchmark       0
ApartYoYChange       0
Date                 0
dtype: int64

In [19]:
mls_data_df.to_csv("./MLS-nulls-dropped.csv", index=False)

In [37]:
mls_data_df['Date'].value_counts()

2018-05-01    57
2017-12-01    57
2017-06-01    57
2017-07-01    57
2017-08-01    57
2017-09-01    57
2017-10-01    57
2017-11-01    57
2018-01-01    57
2018-02-01    57
2018-03-01    57
2018-04-01    57
2019-03-01    56
2018-12-01    56
2018-06-01    56
2018-07-01    56
2018-08-01    56
2019-01-01    56
2019-02-01    56
2019-04-01    56
2018-10-01    56
2019-07-01    56
2019-05-01    56
2018-11-01    56
2019-06-01    56
2019-08-01    54
2019-09-01    54
2019-10-01    54
2019-11-01    54
2019-12-01    54
2020-06-01    54
2020-07-01    54
2020-08-01    54
2020-09-01    54
2020-10-01    54
2020-11-01    54
2020-12-01    54
2021-02-01    54
2020-05-01    54
2021-03-01    54
2018-09-01    54
2020-02-01    53
2020-04-01    53
2020-03-01    53
2020-01-01    53
2021-01-01    52
2017-02-01    50
2017-03-01    50
2016-02-01    50
2016-03-01    50
2016-04-01    50
2016-05-01    50
2017-01-01    50
2017-04-01    50
2017-05-01    50
2015-08-01    50
2016-06-01    50
2016-07-01    50
2016-08-01    

In [36]:
pd.set_option('display.max_rows', 500)

In [38]:
date_values_df = mls_data_df['Date'].value_counts()

In [46]:
date_values_df.sort_index(ascending=True)

2015-07-01    50
2015-08-01    50
2015-09-01    49
2015-10-01    49
2015-11-01    49
2015-12-01    49
2016-01-01    49
2016-02-01    50
2016-03-01    50
2016-04-01    50
2016-05-01    50
2016-06-01    50
2016-07-01    50
2016-08-01    50
2016-09-01    50
2016-10-01    50
2016-11-01    50
2016-12-01    50
2017-01-01    50
2017-02-01    50
2017-03-01    50
2017-04-01    50
2017-05-01    50
2017-06-01    57
2017-07-01    57
2017-08-01    57
2017-09-01    57
2017-10-01    57
2017-11-01    57
2017-12-01    57
2018-01-01    57
2018-02-01    57
2018-03-01    57
2018-04-01    57
2018-05-01    57
2018-06-01    56
2018-07-01    56
2018-08-01    56
2018-09-01    54
2018-10-01    56
2018-11-01    56
2018-12-01    56
2019-01-01    56
2019-02-01    56
2019-03-01    56
2019-04-01    56
2019-05-01    56
2019-06-01    56
2019-07-01    56
2019-08-01    54
2019-09-01    54
2019-10-01    54
2019-11-01    54
2019-12-01    54
2020-01-01    53
2020-02-01    53
2020-03-01    53
2020-04-01    53
2020-05-01    

In [49]:
mls_data_df.set_index(['Location'], inplace=True)
mls_data_df.head()

,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
Location,,,,,,,,,,,,,,,,
Ajax,179.8,444300.0,16.00,178.8,478900.0,17.86,187.1,402400.0,15.64,170.5,311600.0,18.16,148.7,252500.0,5.61,2015-07-01
Ajax,180.4,445800.0,16.31,179.2,480000.0,15.76,187.6,403500.0,16.81,170.4,311400.0,15.76,151.5,257200.0,8.99,2015-08-01
Ajax,182.8,451700.0,15.55,181.4,485800.0,15.10,190.1,408800.0,15.99,172.3,314900.0,14.33,154.3,262000.0,10.53,2015-09-01
Ajax,183.8,454200.0,15.74,182.3,488300.0,14.51,191.2,411200.0,15.95,177.1,323700.0,17.13,152.4,258800.0,8.70,2015-10-01
Ajax,182.1,450000.0,12.69,180.3,482900.0,11.99,190.3,409300.0,13.14,173.1,316300.0,12.99,151.2,256700.0,5.00,2015-11-01


In [50]:
mls_data_df.loc['York Region']

,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
Location,,,,,,,,,,,,,,,,
York Region,204.3,701100.0,13.50,206.3,811000.0,14.36,210.2,609100.0,14.61,184.7,469300.0,11.87,164.4,356700.0,4.31,2015-07-01
York Region,205.9,706600.0,14.14,207.7,816500.0,14.69,211.4,612600.0,14.58,186.8,474700.0,14.04,167.4,363200.0,7.17,2015-08-01
York Region,206.6,709000.0,13.95,208.7,820500.0,14.73,212.2,614900.0,14.70,187.1,475400.0,12.30,166.3,360800.0,5.52,2015-09-01
York Region,208.7,716200.0,14.17,211.1,829900.0,14.79,213.6,618900.0,14.59,189.7,482000.0,14.35,168.1,364700.0,7.07,2015-10-01
York Region,209.2,717900.0,14.50,211.9,833000.0,15.23,214.8,622400.0,14.99,189.4,481300.0,13.89,165.5,359000.0,5.75,2015-11-01
York Region,210.1,721000.0,14.31,213.0,837400.0,14.82,215.5,624400.0,14.57,190.2,483300.0,16.47,165.7,359500.0,6.49,2015-12-01
York Region,212.2,728700.0,14.52,215.2,846200.0,15.20,217.1,629200.0,14.20,188.5,479400.0,13.08,166.6,361400.0,7.48,2016-01-01
York Region,215.5,740000.0,14.08,218.7,859900.0,14.80,220.9,640200.0,13.69,195.1,496100.0,14.56,170.1,368900.0,9.81,2016-02-01
York Region,221.9,762000.0,15.39,225.9,888200.0,16.62,228.2,661300.0,15.66,199.7,507800.0,14.57,171.1,371100.0,7.68,2016-03-01


In [60]:
mls_data_df.reset_index()

,Location,CompIndex,CompBenchmark,CompYoYChange,SFDetachIndex,SFDetachBenchmark,SFDetachYoYChange,SFAttachIndex,SFAttachBenchmark,SFAttachYoYChange,THouseIndex,THouseBenchmark,THouseYoYChange,ApartIndex,ApartBenchmark,ApartYoYChange,Date
0,Ajax,179.8,444300.0,16.00,178.8,478900.0,17.86,187.1,402400.0,15.64,170.5,311600.0,18.16,148.7,252500.0,5.61,2015-07-01
1,Ajax,180.4,445800.0,16.31,179.2,480000.0,15.76,187.6,403500.0,16.81,170.4,311400.0,15.76,151.5,257200.0,8.99,2015-08-01
2,Ajax,182.8,451700.0,15.55,181.4,485800.0,15.10,190.1,408800.0,15.99,172.3,314900.0,14.33,154.3,262000.0,10.53,2015-09-01
3,Ajax,183.8,454200.0,15.74,182.3,488300.0,14.51,191.2,411200.0,15.95,177.1,323700.0,17.13,152.4,258800.0,8.70,2015-10-01
4,Ajax,182.1,450000.0,12.69,180.3,482900.0,11.99,190.3,409300.0,13.14,173.1,316300.0,12.99,151.2,256700.0,5.00,2015-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3680,York Region,287.0,970300.0,11.76,292.0,1112000.0,12.65,293.2,847300.0,12.73,271.3,704100.0,11.28,252.2,562700.0,6.91,2020-11-01
3681,York Region,289.5,978700.0,12.56,295.2,1124200.0,13.89,296.3,856200.0,13.66,270.4,701800.0,10.28,253.2,564900.0,6.84,2020-12-01
3682,York Region,294.9,997000.0,13.34,301.7,1148900.0,15.20,302.9,875300.0,14.78,272.6,707500.0,10.14,254.7,568200.0,5.86,2021-01-01
3683,York Region,309.9,1047700.0,17.16,318.6,1213300.0,20.23,322.6,932200.0,20.15,280.7,728500.0,11.04,260.2,580500.0,5.39,2021-02-01
